# DS100 Final Project : Building a Logistic Classifier for credit default risk in Taiwan

# **Project Partners - Thiha Aung & Julia Chen**

# **1. READ THE DATA**







## (a). Let's import our dataset and take a look at what the columns represents




In [0]:
import pandas as pd
import numpy as np
import altair as alt
import io
from pprint import pprint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.utils import resample

In [0]:
creditdefault = pd.read_excel('default_of_credit_card_clients.xls',skiprows=1)
creditdefault = creditdefault.drop('ID', 1) #Remove first column "ID" 
creditdefault.head()

## (b). Descriptions of each predictors/ regressors



1. LIMIT_BAL: **(numeric)** Amount of given credit in NT dollars 

2. SEX: **(categorical)** Gender (1=male, 2=female)

3. EDUCATION: **(categorical)** (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown) 

4. MARRIAGE: **(categorical)** Marital status (1=married, 2=single, 3=others)

5. AGE: **(numeric)** Age in years 

6. PAY_0, PAY_2-6: **(categorical)** Repayment status from April to September, 2005 
(-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above) 

7. BILL_AMT1-6: **(numeric)** Amount of bill statement in September, July... April, 2005 

8. PAY_AMT1-6: **(numeric)** Amount of previous payment in September, July... April, 2005 

9. default.payment.next.month:  **(predict variable)** Default payment (1=yes, 0=no)



In [0]:
creditdefault.info()

## (c).  Check if there is any null values in the dataset.




In [0]:
creditdefault.isnull().sum()
#NO missing variable

In [0]:
creditdefault.describe()

# **2. DATA CLEANING**

In this process, we will focus on verifying and cleaning categorial variables.



## (a). Check feature "SEX".

In [0]:
#Check each category variable
print(creditdefault['SEX'].unique())


Each variable is documented such that male = 1, female = 0. No action is needed further.

## (b). Check feature "EDUCATION".

In [0]:
print(creditdefault['EDUCATION'].unique())


For feature "Education",value 0 is not documented, value 4 is others 5,6 unknown. Hence, it is reasonable to assume that values 0,4,5,6 are same and set all of them to 4. 

In [0]:
change = (creditdefault['EDUCATION'] == 0
          ) | (creditdefault['EDUCATION'] == 4) | (creditdefault['EDUCATION'] == 5) | (creditdefault['EDUCATION'] == 6)

creditdefault.loc[change,'EDUCATION'] = 4

print(creditdefault['EDUCATION'].unique())

## (c). Check feature "MARRIAGE".

In [0]:
print(creditdefault['MARRIAGE'].unique())


0 IS undocumented and 3 is others. I suggest that both values form a group of people who are either divoreced or doesn't want to answer. Let's set both to 3. 

In [0]:
change = (creditdefault['MARRIAGE'] == 0) | (creditdefault['MARRIAGE'] == 3)
creditdefault.loc[change,'MARRIAGE'] = 3 # set the values of credit default with 0 or 3 to 3
print(creditdefault['MARRIAGE'].unique())

## (d). Check features "PAY_0","PAY2" to "PAY_6"

In [0]:
for i in [0,2,3,4,5,6]:
  string = "PAY_"
  string = string+str(i)
  print(creditdefault[string].unique())

Recall: PAY_0, PAY_2-6: Repayment status from August to September, 2005 
(-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)  

**Note that we have data point -2 which has no description. Since -1 is described as pay duly,we assume -2 is also pay duly or pay one month ahead and set both to 1.**

In [0]:
for i in [0,2,3,4,5,6]:
  string = "PAY_"
  string = string+str(i)
  change = (creditdefault[string] == -2) | (creditdefault[string] == -1)
  creditdefault.loc[change,string] = -1
  print(np.sort(creditdefault[string].unique()))

##  (e) Since EDUCATION feature is ordinal categorial data, we will do one-hot encoding.


In [0]:
# Transform all back to their according names for education
from sklearn.feature_extraction import DictVectorizer

mylist = ["Graduate School", "University","High school", "Others"]
i =1
for x in mylist:
    change = creditdefault['EDUCATION'] == i
    creditdefault.loc[change,'EDUCATION'] = x
    i = i + 1


EDUCATION = creditdefault[['EDUCATION']].to_dict(orient='records')
encoder = DictVectorizer(sparse=False)
newdf = pd.DataFrame(
    data = encoder.fit_transform(EDUCATION),
    columns = encoder.feature_names_
)
creditdefault= pd.concat([creditdefault ,newdf], ignore_index=False, axis=1)
creditdefault.head()

## (f) For visualization interpretability, we will add columns of desriptions of each categorical variables (SEX,MARRIAGE, PAY_0,PAY_2 to PAY_6).  
 For example, for 'SEX' feature, we have integer encoding 1 and 2. We will add a column named 'SEX DESCRIPTION' where 1 is 'Male' and 2 is 'Female'


In [0]:
#Added Description for 'SEX' feature
#1 is male 2 is female
mylist = ["Male","Female"]
i =1
for x in mylist:
    change = creditdefault['SEX'] == i
    creditdefault.loc[change,'SEX DESCRIPTION'] = x
    i = i + 1
    
creditdefault.head()

In [0]:
#Added Description for 'MARRIAGE' feature
mylist = ["Married", "Single","Others"]
i =1
for x in mylist:
    change = creditdefault['MARRIAGE'] == i
    creditdefault.loc[change,'MARRIAGE DESCRIPTION'] = x
    i = i + 1


creditdefault.head()

In [0]:
#Add description for Pay
#PAY_0, PAY_2-6: (categorical) Repayment status from August to September, 2005
# -1=pay duly, 1=payment delay for one month, 2=payment delay for two months, 8=payment delay for eight months, 
# 9=payment delay for nine months and above


mylist = ["Pay Duly","1 Month Delay","2 MonthDelay","3 Month Delay","4 Month Delay","5 Month Delay","6 Month Delay",
          "7 Month Delay","8 Month Delay","9 Month Or More Delay"]
features = {"PAY_0" : 'PAY_0(SEPT STATUS)',"PAY_2":'PAY_2(AUG STATUS)',"PAY_3":'PAY_3(JULY STATUS)',
            "PAY_4":'PAY_4(JUN STATUS)',"PAY_5":'PAY_5(MAY STATUS)',"PAY_6":'PAY_6(APR STATUS)'}
mydict  = { -1 : "Pay Duly", 0: "1 Month Delay",1: "2 MonthDelay",2: "3 Month Delay",3: "4 Month Delay",4:"5 Month Delay",
           5:"6 Month Delay",6:"7 Month Delay",7:"8 Month Delay",8:"9 Month Or More Delay"}
for key in features:
  for x in mylist:
    for i in np.sort(creditdefault[key].unique()):
      change = creditdefault[key] == i
      creditdefault.loc[change,features.get(key)] = mydict.get(i)

creditdefault.head()


# **3. EXPLORATORY DATA ANALYSIS** 

# (a). Exploration based on the Y (default payment next month)

In [0]:
creditdefault.groupby('default payment next month').mean()

* We could observed that the Limit Balance for no default payment next 
month is much higher than default payment next month. 

* Overall, Sex and Education and Marriage status are relatively same whatever if there is a default payment next month or not.

* Repayment status from April to September is highly correlated with default payment, if there is no default payment, the repayment status will be negative which means it is pay duly.

* Amount of bill statement of no efault payment next month is slightly higher than default payment next month.

* Amount of previous payment of no efault payment next month is significantly higher than default payment next month.

Indeed, we could groupby other categorical variable and visualize such as Sex, Education, Marital status to get a more detailed sense of our data.



# (b) Some Visualization


#### **(i) Sex**


In [0]:
alt.data_transformers.disable_max_rows()
source = creditdefault[creditdefault['default payment next month']==1]
alt.Chart(source).mark_bar().encode(
    x='SEX DESCRIPTION:O',
    y='count(default payment next month):Q',
    column='MARRIAGE DESCRIPTION:O'
).properties(width=220)

# more female in single seems to no default

In [0]:
# Defined a pd series that finds the relationship between each feature 
# and default or no and the percent
def correlationframe(Col1, Col2):
    result = creditdefault.groupby([Col1, Col2]).size().unstack()
    result['Default percent'] = (result[result.columns[1]]/(result[result.columns[0]] + result[result.columns[1]]))
    return result
correlationframe(creditdefault['SEX DESCRIPTION'],creditdefault['default payment next month'])

Note more females (2) in data but more males default (24 percent)! Interesting...

In [0]:
source = creditdefault
alt.Chart(source).mark_bar().encode(
    x='SEX DESCRIPTION:O',
    y='count(default payment next month):Q',
    column='default payment next month:O',
    color='default payment next month:O'
).properties(width=220)

The total numbers are similar for married and single. But married people default more! 

#### **(ii) Education**


In [0]:
correlationframe(creditdefault['EDUCATION'],creditdefault['default payment next month'])

More people in high school and university default more compared to people who go to graduate school. 


In [0]:
source = creditdefault
alt.Chart(source).mark_bar().encode(
    x='EDUCATION:O',
    y='count(default payment next month):Q',
    column='default payment next month:O',
    color='default payment next month:O'
).properties(width=220)

For people who atttend university and graduate school wi

#### **(iii) PAY_0 (Last Payment Satus)**



In [0]:
source = creditdefault
alt.Chart(source).mark_bar().encode(
    x='PAY_0(SEPT STATUS):O',
    y='count(default payment next month):Q',
    column='default payment next month:O',
    color='default payment next month:O'
).properties(width=220)

#### **(iv) Marriage**


In [0]:
correlationframe(creditdefault['MARRIAGE DESCRIPTION'],creditdefault['default payment next month'])

In [0]:
source = creditdefault
alt.Chart(source).mark_bar().encode(
    x='MARRIAGE DESCRIPTION:O',
    y='count(default payment next month):Q',
    column='default payment next month:O',
    color='default payment next month:O'
).properties(width=220)

#### (v) Age

In [0]:
source = creditdefault[creditdefault['default payment next month']==1]
alt.Chart(source).mark_bar().encode(
    alt.X("AGE:O",bin = True),
    y='count(AGE)',
    column = 'MARRIAGE DESCRIPTION'
).properties(width=230)

# 20-30people default the most among singles
# married people default around 30-50

#### (vi) Limit Balance

In [0]:
alt.Chart(creditdefault).mark_boxplot().encode(
    alt.X("AGE:O",bin = True),
    alt.Y('LIMIT_BAL:Q',title = "Limit balance") 
).properties(width=230)


In [0]:
alt.Chart(creditdefault).mark_boxplot().encode(
    alt.X("SEX DESCRIPTION:N"),
    alt.Y('LIMIT_BAL:Q',title = "Limit Balance")
).properties(width=230)


In [0]:
source = creditdefault
alt.data_transformers.disable_max_rows()

alt.Chart(source).mark_bar(size=40).encode(
    x='SEX DESCRIPTION:O',
    y='count(default payment next month):Q',
    #column='default payment next month:O',
    color='default payment next month:O'
).properties(width=250,height=200)

In [0]:
alt.Chart(source).mark_bar(size=40).encode(
    x='EDUCATION:O',
    y='count(default payment next month):Q',
    #column='default payment next month:O',
    color='default payment next month:O'
).properties(width=250,height=200)

In [0]:
alt.Chart(source).mark_bar(size=40).encode(
    x='MARRIAGE DESCRIPTION:O',
    y='count(default payment next month):Q',
    #column='default payment next month:O',
    color='default payment next month:O'
).properties(width=250,height=200)

# **4. PCA**




In [0]:
data=creditdefault[['LIMIT_BAL','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                       'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']]
corrMatrix = data.corr().reset_index().melt('index')
corrMatrix.columns = ['var1', 'var2', 'correlation']
base = alt.Chart(corrMatrix).transform_filter(
    alt.datum.var1 < alt.datum.var2
).encode(
    x='var1',
    y='var2',
).properties(
    width=alt.Step(30),
    height=alt.Step(30)
)

rects = base.mark_rect(size=30).encode(
    color='correlation'
)

text = base.mark_text(
    size=10
).encode(
    text=alt.Text('correlation', format=".2f"),
    color=alt.condition(
        "datum.correlation > 0.5",
        alt.value('white'),
        alt.value('black')
    )
)
rects + text

In [0]:
def winsorize(data, variable, k=0):
    # Sort the values of the given variable to find the replacement values
    sorted_val = np.sort(data[variable])
    total = len(data)
    lower = sorted_val[k]
    upper = sorted_val[total-k-1]
    
    # Make a copy() of the data so not to change the original dataframe
    data_copy = data

    ## Replace outliers on data_copy
    data_copy.loc[(data_copy[variable] < lower), variable] = lower
    data_copy.loc[(data_copy[variable] > upper), variable] = upper
    
    return data_copy

In [0]:
creditdefault=winsorize(creditdefault, 'PAY_AMT1', k=100)
creditdefault=winsorize(creditdefault, 'PAY_AMT2', k=100)
creditdefault=winsorize(creditdefault, 'PAY_AMT3', k=100)
creditdefault=winsorize(creditdefault, 'PAY_AMT4', k=100)
creditdefault=winsorize(creditdefault, 'PAY_AMT5', k=100)
creditdefault=winsorize(creditdefault, 'PAY_AMT6', k=100)

creditdefault=winsorize(creditdefault, 'BILL_AMT1', k=100)
creditdefault=winsorize(creditdefault, 'BILL_AMT2', k=100)
creditdefault=winsorize(creditdefault, 'BILL_AMT3', k=100)
creditdefault=winsorize(creditdefault, 'BILL_AMT4', k=100)
creditdefault=winsorize(creditdefault, 'BILL_AMT5', k=100)
creditdefault=winsorize(creditdefault, 'BILL_AMT6', k=100)

First of all, balanced default payment next month

In [0]:
y = creditdefault[['default payment next month']]
X = creditdefault.loc[:, creditdefault.columns != 'default payment next month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
not_default = X[X['default payment next month']==0]
default = X[X['default payment next month']==1]

default_upsampled  = resample(default, replace=True, # sample with replacement
               n_samples=len(not_default)) # match number in majority class
balanced = pd.concat([not_default, default_upsampled])

Check balance

In [0]:
len(balanced)
balanced['default payment next month'].value_counts()

In [0]:
PAY_BILL_AMT=balanced[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','PAY_AMT6',
                        'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']]
defaultPayment=balanced[['default payment next month']]

#center the PAY_BILL_AMT
n = PAY_BILL_AMT.shape[0]
mean = np.mean(PAY_BILL_AMT, axis = 0)
sd = (np.var(PAY_BILL_AMT, axis = 0))**(1/2)
normalized_AMT= (PAY_BILL_AMT - mean)/(sd)/n**(1/2)

my_dict=np.linalg.svd(normalized_AMT, full_matrices=False)
u = my_dict[0]
s = my_dict[1]
vt = my_dict[2]
first_two_vt_vectors = vt[:2] 
x_2d = normalized_AMT @ first_two_vt_vectors.T

Visualize first 2 PCs

In [0]:
alt.data_transformers.disable_max_rows()

plot_pca = pd.DataFrame({
    'x': x_2d.iloc[:,0],
    'y': x_2d.iloc[:,1],
     'color': defaultPayment['default payment next month']
})


alt.Chart(plot_pca).mark_circle(size=5).encode(
    x = alt.X('x', title=' PC1'),
    y = alt.Y('y', title=' PC2'),
    #color =  'default payment next month:O'
    color = alt.Color('color:N', scale=alt.Scale(scheme='set1'))
)
#Note: remove outlier for BILL AND PAY

In [0]:
explained = pd.DataFrame({
    'PC #': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Fraction of Variance Explained' : np.square(s[:10])
})

# Draw visualization
alt.Chart(explained).mark_bar(size=20).encode(
    alt.X('PC #:O'),
    alt.Y('Fraction of Variance Explained:Q')
).configure_axisX(labelAngle=0
).properties(width=400)


Since principal component matrix  $P$ is simply the original data rotated in space so that it appears axis aligned. We could just use $P=XV$, From the plot, Variance Explained by Principal Components, there is a cutoff at PC7, so will use first 7 PCs.

In [0]:
pcs = normalized_AMT @ vt.T
pcs = pcs.rename(columns = {0: "pc1", 1: "pc2", 2: "pc3", 3: "pc4", 4: "pc5", 5: "pc6", 6: "pc7",
                            7: "pc8", 8: "pc9", 9: "pc10", 10: "pc11", 11: "pc12"})
pcs.iloc[:,[0,1,2,3,4,5,6,7]]

Prediction based on the 7 PCs

In [0]:
y = defaultPayment
X = pcs.iloc[:,[0,1,2,3,4,5,6]]

lrmodel= LogisticRegression(penalty='l1',C=0.01,solver='liblinear')
lrmodel.fit(X,y.values.ravel())

Logistic Regression Model Fitting

In [0]:
X=X_test[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','PAY_AMT6',
                        'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']]
y_test

#center the X
n = X.shape[0]
mean = np.mean(X, axis = 0)
sd = (np.var(X, axis = 0))**(1/2)
normalized_AMT= (X - mean)/(sd)/n**(1/2)

my_dict=np.linalg.svd(normalized_AMT, full_matrices=False)
u = my_dict[0]
s = my_dict[1]
vt = my_dict[2]


pcs_test = normalized_AMT @ vt.T
pcs_test = pcs_test.rename(columns = {0: "pc1", 1: "pc2", 2: "pc3", 3: "pc4", 4: "pc5", 5: "pc6", 6: "pc7",
                            7: "pc8", 8: "pc9", 9: "pc10", 10: "pc11", 11: "pc12"})
pcs_test.iloc[:,[0,1,2,3,4,5,6]]

lrmodel.fit(pcs_test.iloc[:,[0,1,2,3,4,5,6]],y_test.values.ravel())
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lrmodel.score(pcs_test.iloc[:,[0,1,2,3,4,5,6]],y_test)))


# **5. Feature Engineering**


Based on EDA above, We will create two new features such that one feature will add the effect of Limit Balance on Sex and another will be the effect of Limit Balance on Age since limit balance can influence these features.




In [0]:
creditdefault['LIMIT & SEX'] = creditdefault['LIMIT_BAL'] * creditdefault['SEX']
creditdefault['LIMIT & AGE'] = creditdefault['LIMIT_BAL'] * creditdefault['AGE']
creditdefault.head()

# **6. LOGISTIC MODEL** 

In credit risk analysis, 
type I error( Predict customers default(1) given they don't default(0))   
is less important than type II errors(Predict customers don't default(0) given they default(1)). The more type II error, the more loss the banks receive. Hence, we want a model with lesst type II error.



### (a) Check data imbalance

In [0]:
creditdefault.groupby('default payment next month').count()['LIMIT_BAL']

### (b) Oversample and Undersample 

In [0]:
from sklearn.model_selection import train_test_split
y = creditdefault[['default payment next month']]

X = creditdefault.loc[:,['LIMIT_BAL', 'SEX', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'EDUCATION=Graduate School',
       'EDUCATION=High school', 'EDUCATION=Others','EDUCATION=University','LIMIT & SEX','LIMIT & AGE']]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=.20)

In [0]:
df_train = X_train.join(y_train)
df_train.groupby('default payment next month').count()[['LIMIT_BAL']]
x = df_train.groupby('default payment next month').count()[['LIMIT_BAL']].iloc[0,0]
y = df_train.groupby('default payment next month').count()[['LIMIT_BAL']].iloc[1,0]
print(x)
print(y)
df_train.groupby('default payment next month').count()[['LIMIT_BAL']]

In [0]:
from sklearn.utils import resample
df_majority = df_train[df_train['default payment next month']==0]
df_minority = df_train[df_train['default payment next month']==1]

In [0]:
# Oversample
df_minority_upsampled = resample(df_minority,replace=True,n_samples=x) 
oversampletrain  = pd.concat([df_majority, df_minority_upsampled])
oversampletrain.groupby('default payment next month').count()[['LIMIT_BAL']]

There is risk of overfitting for oversampling.


In [0]:
# Undersample
# we dont need replacement since we have more than enough
df_majority_undersampled = resample(df_majority,replace=False,n_samples=y) 
undersampletrain  = pd.concat([df_majority_undersampled, df_minority])
undersampletrain.groupby('default payment next month').count()[['LIMIT_BAL']]

There is risk of removing useful rows and adding bias.

### (c) Model with oversampling

In [0]:
y_train1 = oversampletrain[['default payment next month']]
X_train1 = oversampletrain.loc[:, ['LIMIT_BAL', 'SEX', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'EDUCATION=Graduate School',
       'EDUCATION=High school', 'EDUCATION=Others','EDUCATION=University','LIMIT & SEX','LIMIT & AGE']]

lrmodel= LogisticRegression(penalty='l1',C=0.01,solver='liblinear')
result=lrmodel.fit(X_train1,y_train1)
y_fitted = lrmodel.predict(X_train1)
y_predicted = lrmodel.predict(X_test)
from  sklearn import model_selection
#3fold cross validation using whole dataset before split or any sampling
scores = model_selection.cross_validate(lrmodel, pd.concat([X_train,X_test]), pd.concat([y_train,y_test]), cv=5, return_train_score=True)
print('Train scores:')
print(scores['train_score'])
print('Test scores:')
print(scores['test_score'])
trainsc = np.round(np.mean(scores['train_score']),3)
testsc = np.round(np.mean(scores['test_score']),3)
df = pd.DataFrame({"Training":[trainsc] ,"Testing":[testsc]})
df = df.rename(index={0: "Accuracy"})
df

In [0]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [0]:
from sklearn.metrics import confusion_matrix
mydf1 = confusion_matrix(y_test, y_predicted)
print(mydf1)
myconfusion1 = pd.DataFrame({"Predict Not Default":mydf1[:,0],"Predict Default ":mydf1[:,1]})
myconfusion1 = myconfusion1.rename(index={0: "Actual Not Default",1:"Actual Default"})
myconfusion1

### (d) Model with undersampling

In [0]:
y_train2 = undersampletrain[['default payment next month']]
X_train2 = undersampletrain.loc[:, ['LIMIT_BAL', 'SEX', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'EDUCATION=Graduate School',
       'EDUCATION=High school', 'EDUCATION=Others','EDUCATION=University','LIMIT & SEX','LIMIT & AGE']]

lrmodel= LogisticRegression(penalty='l1',C=0.01,solver='liblinear')
result=lrmodel.fit(X_train2,y_train2)
y_fitted = lrmodel.predict(X_train2)
y_predicted = lrmodel.predict(X_test)

#3fold cross validation using whole dataset before split or any sampling
scores = model_selection.cross_validate(lrmodel,pd.concat([X_train,X_test]), pd.concat([y_train,y_test]), cv=5, return_train_score=True)
print('Train scores:')
print(scores['train_score'])
print('Test scores:')
print(scores['test_score'])
trainsc = np.round(np.mean(scores['train_score']),3)
testsc = np.round(np.mean(scores['test_score']),3)
df = pd.DataFrame({"Training":[trainsc] ,"Testing":[testsc]})
df = df.rename(index={0: "Accuracy"})
df

In [0]:

# C00 predict 0 given 0, C01 predict 1(default) given they don't default(0)
mydf2 = confusion_matrix(y_test, y_predicted)
print(mydf2)
myconfusion2 = pd.DataFrame({"Predict Not Default":mydf2[:,0],"Predict Default ":mydf2[:,1]})
myconfusion2 = myconfusion2.rename(index={0: "Actual Not Default",1:"Actual Default"})
myconfusion2

### (e) Model without undersampling or oversampling

In [0]:
lrmodel= LogisticRegression(penalty='l1',C=0.01,solver='liblinear')
result=lrmodel.fit(X_train,y_train)
y_fitted = lrmodel.predict(X_train)
y_predicted = lrmodel.predict(X_test)


#3fold cross validation using whole dataset before split or any sampling
scores = model_selection.cross_validate(lrmodel, pd.concat([X_train,X_test]), pd.concat([y_train,y_test]), cv=5, return_train_score=True)
print('Train scores:')
print(scores['train_score'])
print('Test scores:')
print(scores['test_score'])
trainsc = np.round(np.mean(scores['train_score']),3)
testsc = np.round(np.mean(scores['test_score']),3)
df = pd.DataFrame({"Training":[trainsc] ,"Testing":[testsc]})
df = df.rename(index={0: "Accuracy"})

print("Table 1. Training and Testing Accuracy of Simple Logistic model")
df

In [0]:
mydf3 = confusion_matrix(y_test, y_predicted)
print(mydf3)
myconfusion3 = pd.DataFrame({"Predict Not Default":mydf3[:,0],"Predict Default ":mydf3[:,1]})
myconfusion3 = myconfusion3.rename(index={0: "Actual Not Default",1:"Actual Default"})
myconfusion3

Note without oversampling or undersampling, the model has a lot of type II errors i.e. predict not default given actual default.  We want a model with least type II errors or type II errors less than type I error. Hence, we will use model with undersampling.

# 8 Feature selection from undersampled data using Anova

In [0]:
anova_table = sm.Logit(y_train,X_train)
print(anova_table.fit().summary())

# 9 Model with undersampling after feature selection

In [0]:
X_train2 = undersampletrain.loc[:, ['LIMIT_BAL',  'MARRIAGE',  'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4','BILL_AMT1','BILL_AMT2', 'PAY_AMT1',
       'PAY_AMT2',  'PAY_AMT4','EDUCATION=Graduate School',
       'EDUCATION=High school', 'EDUCATION=Others','EDUCATION=University','LIMIT & AGE']]
X_test2 = X_test.loc[:, ['LIMIT_BAL',  'MARRIAGE',  'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4','BILL_AMT1','BILL_AMT2', 'PAY_AMT1',
       'PAY_AMT2',  'PAY_AMT4','EDUCATION=Graduate School',
       'EDUCATION=High school', 'EDUCATION=Others','EDUCATION=University','LIMIT & AGE']]
lrmodel= LogisticRegression(penalty='l1',C=0.01,solver='liblinear')
result=lrmodel.fit(X_train2,y_train2)
y_fitted = lrmodel.predict(X_train2)
y_predicted = lrmodel.predict(X_test2)

from  sklearn import model_selection
#3fold cross validation using whole dataset before split or any sampling
scores = model_selection.cross_validate(lrmodel, pd.concat([X_train,X_test]), pd.concat([y_train,y_test]), cv=5, return_train_score=True)
print('Train scores:')
print(scores['train_score'])
print('Test scores:')
print(scores['test_score'])
trainsc = np.round(np.mean(scores['train_score']),3)
testsc = np.round(np.mean(scores['test_score']),3)
df = pd.DataFrame({"Training":[trainsc] ,"Testing":[testsc]})
df = df.rename(index={0: "Accuracy"})
df

In [0]:
mydf4 = confusion_matrix(y_test, y_predicted)
print(mydf4)
myconfusion4 = pd.DataFrame({"Predict Not Default":mydf4[:,0],"Predict Default ":mydf4[:,1]})
myconfusion4 = myconfusion4.rename(index={0: "Actual Not Default",1:"Actual Default"})
myconfusion4

Overall, with features removed using ANOVA , the reduced model gets same predictive power as full model both undersampled. However, reduced model has a balanced type I and type II error whereas full model has much less type II error than type I error. Both model are undersampled. Based on business model, we can use either of the model to open credit for new customers. 

In [0]:
precision = mydf4[1,1]/np.sum(mydf4[1,1]+ mydf4[0,1]) # BIGGER TYPE I ERROR, smaller precision 
recall = mydf4[1,1]/np.sum(mydf4[1,1]+ mydf4[1,0]) # bigger type II (FN), smaller recall, better recall smaller type II

precisionlist = [mydf1[1,1]/np.sum(mydf1[1,1]+ mydf1[0,1]),mydf2[1,1]/np.sum(mydf2[1,1]+ mydf2[0,1]),
                mydf3[1,1]/np.sum(mydf3[1,1]+ mydf3[0,1]),mydf4[1,1]/np.sum(mydf4[1,1]+ mydf4[0,1])]
                                   
precisionlist     = np.round(precisionlist,4)       
precisionlist

recallist = [mydf1[1,1]/np.sum(mydf1[1,1]+ mydf1[1,0]),mydf2[1,1]/np.sum(mydf2[1,1]+ mydf2[1,0]),
                mydf3[1,1]/np.sum(mydf3[1,1]+ mydf3[1,0]),mydf4[1,1]/np.sum(mydf4[1,1]+ mydf4[1,0])]
recallist


prenrec = pd.DataFrame({"Precision":precisionlist,"Recall":recallist})
prenrec = prenrec.rename(index={0: "Oversample",1:"Undersample",2: "Simple ",3:"Undersample with feature selection"})
prenrec